In [10]:
import numpy as np
import glob, tqdm
import sys, dill
sys.path.append('../src')
from lib.utils import *

In [2]:
def sort_fn(filename):
    date_string = filename[-14:-4]
    datetime_object = datetime.datetime.strptime(date_string, "%Y-%m-%d")
    return datetime_object


data_dir = "/data/allen/climate_data/"
x_paths = sorted(glob.glob(f"{data_dir}/CAM5/inputs*"), key=sort_fn)
y_paths = sorted(glob.glob(f"{data_dir}/CAM5/outputs*"), key=sort_fn)

In [13]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

for i, (x_f, y_f) in tqdm.tqdm(enumerate(zip(x_paths, y_paths)), total=len(y_paths)):
    x_time, y_time = x_f[-14:-4], y_f[-14:-4]

    x = np.load(x_f, mmap_mode='r')
    y = np.load(y_f, mmap_mode='r')
    x_scaler.fit(x)
    y_scaler.fit(y)


def save_object(obj, filename):
    dill.dump(obj, file=open(filename, "wb"))

save_object(x_scaler, f"x_SPCAM5_minmax_scaler.dill")
save_object(y_scaler, f"y_SPCAM5_minmax_scaler.dill")

100%|██████████| 727/727 [04:01<00:00,  3.01it/s]


In [4]:
x_scaler = dill.load(open(f"x_CAM5_minmax_scaler.dill", 'rb'))
y_scaler = dill.load(open(f"y_CAM5_minmax_scaler.dill", 'rb'))

In [8]:
x_scaler.max

array([2.77434601e+02, 2.69584564e+02, 2.64400452e+02, 2.60852417e+02,
       2.54367340e+02, 2.50192810e+02, 2.46686691e+02, 2.45020264e+02,
       2.42068619e+02, 2.39250793e+02, 2.39442535e+02, 2.40342361e+02,
       2.42080078e+02, 2.45708694e+02, 2.45818939e+02, 2.54548615e+02,
       2.63722168e+02, 2.71797302e+02, 2.77868713e+02, 2.84104828e+02,
       2.92566071e+02, 3.02001953e+02, 3.10066620e+02, 3.16288116e+02,
       3.20230011e+02, 3.22339264e+02, 1.88973854e-06, 1.87160617e-06,
       1.88490878e-06, 1.92133439e-06, 1.97712529e-06, 2.19409480e-06,
       6.08398796e-06, 4.12740046e-04, 7.71799008e-04, 1.10595103e-03,
       1.47442066e-03, 1.71812484e-03, 2.26741494e-03, 2.36729882e-03,
       2.95128394e-03, 3.53110977e-03, 4.99498053e-03, 6.79808948e-03,
       1.00738229e-02, 1.22102397e-02, 1.36217745e-02, 1.62387900e-02,
       1.92315802e-02, 2.21411064e-02, 2.42159814e-02, 2.76107732e-02,
       0.00000000e+00, 0.00000000e+00, 1.82674378e-27, 2.64542116e-27,
      

In [14]:
x = np.load(x_paths[0])
x = x_scaler.transform(x)

In [15]:
x.min(), x.max()

(0.0, 0.96887195)

In [19]:
y = np.load(y_paths[2])
y = y_scaler.transform(y)

In [10]:
len(x_paths)//2 + 2

365

In [16]:
x_paths[int(365*0.8):]

['/data/allen/climate_data//SPCAM5/inputs_2004-01-18.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-19.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-20.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-21.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-22.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-23.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-24.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-25.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-26.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-27.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-28.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-29.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-30.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-01-31.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-02-01.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-02-02.npy',
 '/data/allen/climate_data//SPCAM5/inputs_2004-02-03.npy

In [20]:
y.min(), y.max()

(0.0, 1.0)